# Chapter 3: Keras and Data Retrieval in TensorFlow 2

<table align="left">
    <td>
        <a target="_blank" href="https://colab.research.google.com/github/thushv89/manning_tf2_in_action/blob/master/Ch03-Keras-and-Data-Retrieval/3.2.Creating_Input_Pipelines.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
    </td>
</table>

## Importing necessary libraries and some setup

In [1]:
import os
import random
import numpy as np
import requests
from zipfile import ZipFile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.models import Sequential
import pandas as pd
import tensorflow_datasets as tfds

def fix_random_seed(seed):
    try:
        np.random.seed(seed)
    except NameError:
        print("Warning: Numpy is not imported. Setting the seed for Numpy failed.")
    try:
        tf.random.set_seed(seed)
    except NameError:
        print("Warning: TensorFlow is not imported. Setting the seed for TensorFlow failed.")
    try:
        random.seed(seed)
    except NameError:
        print("Warning: random module is not imported. Setting the seed for random failed.")

# Fixing the random seed
fix_random_seed(4321)
print("TensorFlow version: {}".format(tf.__version__))

data_dir = 'data'

if not os.path.exists(data_dir):
    os.makedirs(data_dir)


TensorFlow version: 2.9.0


## Using the `tf.data` API to retrieve data

Here we will be using the `tf.data` API to feed a dataset containing images of flowers. The dataset has a folder containing the images and a CSV file listing filenames and their corresponding label as an integer. We will write a TensorFlow data pipeline that does the following.

* Extract filenames and classes from the CSV
* Read in the images from the extracted filenames and resize them to 64x64
* Convert the class labels to one-hot encoded vectors
* Combine the processed images and one-hot encoded vectors to a single dataset
* Finally, shuffle the data and output as batches

### Downloading the data
The dataset is available at https://www.kaggle.com/olgabelitskaya/flower-color-images/data . 

You need to download the zip file available in this URL and place it in the `data` folder in the `Ch03-Keras-and-Data-Retrieval` folder. You **do not** need to extract it as the following code will do it for you.

In [6]:
# Section 3.2

import os
from zipfile import ZipFile

# Extracting the flowers image data to a directory
zip_filepath = [os.path.join('data',f) for f in os.listdir('data') if f.endswith('.zip')]

if len(zip_filepath)==0:
    print("Did you download the dataset as a zip file and place it in the Ch03-Keras-and-Data-Retrieval/data folder?")
elif len(zip_filepath)>1:
    print("There's too many .zip files. There should be only 1")

zfile = ZipFile(zip_filepath[0])
zfile.extractall('data')

## Creating a tf.data.Dataset 

Here we are creating the `tf.data` pipeline that executes the above steps.

In [9]:
# Section 3.2
# Code listing 3.5

import tensorflow as tf
import os
import tensorflow.keras.backend as K

K.clear_session() # Making sure we are clearing out the TensorFlow graph

# Read the CSV file with TensorFlow
# The os.path.sep at the end is important for the get_image function
data_dir = os.path.join('data', 'flower_images', 'flower_images') + os.path.sep
assert os.path.exists(data_dir)
csv_ds = tf.data.experimental.CsvDataset(
    os.path.join(data_dir,'flower_labels.csv') , ("",-1), header=True
)
# Separate the image names and labels to two separate sets
fname_ds = csv_ds.map(lambda a,b: a)
label_ds = csv_ds.map(lambda a,b: b)

def get_image(file_path):
    
    img = tf.io.read_file(data_dir + file_path)
    # convert the compressed string to a 3D uint8 tensor
    img = tf.image.decode_png(img, channels=3)
    # Use `convert_image_dtype` to convert to floats in the [0,1] range.
    img = tf.image.convert_image_dtype(img, tf.float32)
    # resize the image to the desired size.
    return tf.image.resize(img, [64, 64])

# Get the images by running get_image across all the filenames
image_ds = fname_ds.map(get_image)
print("The image dataset contains: {}".format(image_ds))
# Create onehot encoded labels from label data
label_ds = label_ds.map(lambda x: tf.one_hot(x, depth=10))
# Zip the images and labels together
data_ds = tf.data.Dataset.zip((image_ds, label_ds))

# Shuffle the data so that we get a mix of labels in every batch
data_ds = data_ds.shuffle(buffer_size= 20)
# Define a batch of size 5 
data_ds = data_ds.batch(5)


The image dataset contains: <MapDataset element_spec=TensorSpec(shape=(64, 64, 3), dtype=tf.float32, name=None)>


In [10]:
# Iterate through the data to see what it contains
for item in data_ds:
    print(item)
    break

(<tf.Tensor: shape=(5, 64, 64, 3), dtype=float32, numpy=
array([[[[0.02941177, 0.02941177, 0.02745098],
         [0.04117648, 0.05      , 0.03921569],
         [0.14019608, 0.17843139, 0.11176471],
         ...,
         [0.0254902 , 0.0254902 , 0.0254902 ],
         [0.0254902 , 0.0254902 , 0.0254902 ],
         [0.02450981, 0.02450981, 0.02450981]],

        [[0.04411765, 0.04901961, 0.0382353 ],
         [0.13333334, 0.16960785, 0.10490197],
         [0.19901963, 0.2735294 , 0.15490197],
         ...,
         [0.0254902 , 0.0254902 , 0.0254902 ],
         [0.02647059, 0.02647059, 0.02647059],
         [0.02647059, 0.02745098, 0.0254902 ]],

        [[0.12254903, 0.1637255 , 0.09509805],
         [0.20686276, 0.2901961 , 0.15490197],
         [0.1892157 , 0.27549022, 0.14803922],
         ...,
         [0.02843137, 0.02941177, 0.02843137],
         [0.02843137, 0.03333334, 0.02843137],
         [0.0382353 , 0.04411765, 0.03431373]],

        ...,

        [[0.88039225, 0.6186274 , 0

### Defining and training a model

Here we are defining a simple Convolution Neural Network (CNN) model to train it on the image data we just retrieved. You don't have to worry about the technical details of CNNs right now. We will discuss them in detail in the next chapter.

In [11]:
# Section 3.2

from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.models import Sequential

# Defining a Convolution neural network for you to train for the flowers data
# We will discuss convolution neural networks in more detail later
model = Sequential([
    Conv2D(64,(5,5), activation='relu', input_shape=(64,64,3)),
    Flatten(),
    Dense(10, activation='softmax')
])

# Compiling the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

# Training the model with the tf.data pipeline
model.fit(data_ds, epochs=10)

Epoch 1/10


2022-07-29 18:36:00.740818: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8400
2022-07-29 18:36:02.055071: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 11.0.194, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


42/42 [==============================] - 3s 9ms/step - loss: 3.1753 - acc: 0.2810
Epoch 2/10
42/42 [==============================] - 0s 3ms/step - loss: 1.0638 - acc: 0.7000
Epoch 3/10
42/42 [==============================] - 0s 3ms/step - loss: 0.5444 - acc: 0.8762
Epoch 4/10
42/42 [==============================] - 0s 3ms/step - loss: 0.2067 - acc: 0.9429
Epoch 5/10
42/42 [==============================] - 0s 3ms/step - loss: 0.0588 - acc: 0.9952
Epoch 6/10
42/42 [==============================] - 0s 3ms/step - loss: 0.0458 - acc: 0.9905
Epoch 7/10
42/42 [==============================] - 0s 3ms/step - loss: 0.0412 - acc: 0.9952
Epoch 8/10
42/42 [==============================] - 0s 3ms/step - loss: 0.0167 - acc: 0.9952
Epoch 9/10
42/42 [==============================] - 0s 3ms/step - loss: 0.0150 - acc: 0.9952
Epoch 10/10
42/42 [==============================] - 0s 3ms/step - loss: 0.0040 - acc: 1.0000


## Using Keras data generators to retrieve data

Instead of `tf.data` API let us use the Keras `ImageDataGenerator` to retrieve the data. As you can see, the `ImageDataGenerator` involves much less code than the using the `tf.data` API. 

In [12]:
# Section 3.2
# Code listing 3.6

from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import pandas as pd

data_dir = os.path.join('data','flower_images', 'flower_images')

# Defining an image data generator provided in Keras
img_gen = ImageDataGenerator()

# Reading the CSV files containing filenames and labels
labels_df = pd.read_csv(os.path.join(data_dir, 'flower_labels.csv'), header=0)

# Generating data using the flow_from_dataframe function
gen_iter = img_gen.flow_from_dataframe(
    dataframe=labels_df, directory=data_dir, x_col='file', y_col='label', class_mode='raw', batch_size=5, target_size=(64,64))

# Iterating through the data
for item in gen_iter:
    print(item)
    break

Found 210 validated image filenames.
(array([[[[  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         ...,
         [  0.,  74.,   0.],
         [ 15.,  60.,   0.],
         [ 65.,  37.,   0.]],

        [[ 11.,  11.,  11.],
         [ 11.,  11.,  11.],
         [ 25.,  23.,  20.],
         ...,
         [ 62., 109.,  50.],
         [ 63., 113.,  52.],
         [ 65.,  52.,  45.]],

        [[ 13.,  14.,  13.],
         [ 16.,  17.,  15.],
         [ 13.,  15.,  13.],
         ...,
         [ 61., 109.,  48.],
         [ 66., 114.,  52.],
         [ 71.,  86.,  51.]],

        ...,

        [[100.,  83.,  72.],
         [104.,  89.,  75.],
         [ 93.,  80.,  66.],
         ...,
         [ 89.,  83.,  71.],
         [ 94.,  81.,  67.],
         [ 14.,  17.,  18.]],

        [[ 92.,  80.,  64.],
         [115., 101.,  85.],
         [ 68.,  61.,  53.],
         ...,
         [ 80.,  71.,  57.],
         [ 75.,  67.,  56.],
         [ 56.,  49.,  44.]],

## Using the `tensorflow-datasets` library

Here we will use the `tensorflow-datasets` package. It is a curated list of popular datasets available for machine learning projects. With this package you can download a dataset in a single line. This means you don't have to worry about downloading/extracting/formatting data manually. All of that will be already done when you import data using the `tensorflow-datasets` library.

### Lists the available datasets

In [13]:
# Section 3.2

import tensorflow_datasets as tfds
import tensorflow as tf
# See all registered datasets
tfds.list_builders()

2022-07-29 18:36:08.837006: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


['abstract_reasoning',
 'accentdb',
 'aeslc',
 'aflw2k3d',
 'ag_news_subset',
 'ai2_arc',
 'ai2_arc_with_ir',
 'amazon_us_reviews',
 'anli',
 'answer_equivalence',
 'arc',
 'asqa',
 'asset',
 'assin2',
 'bair_robot_pushing_small',
 'bccd',
 'beans',
 'bee_dataset',
 'beir',
 'big_patent',
 'bigearthnet',
 'billsum',
 'binarized_mnist',
 'binary_alpha_digits',
 'ble_wind_field',
 'blimp',
 'booksum',
 'bool_q',
 'c4',
 'caltech101',
 'caltech_birds2010',
 'caltech_birds2011',
 'cardiotox',
 'cars196',
 'cassava',
 'cats_vs_dogs',
 'celeb_a',
 'celeb_a_hq',
 'cfq',
 'cherry_blossoms',
 'chexpert',
 'cifar10',
 'cifar100',
 'cifar10_1',
 'cifar10_corrupted',
 'citrus_leaves',
 'cityscapes',
 'civil_comments',
 'clevr',
 'clic',
 'clinc_oos',
 'cmaterdb',
 'cnn_dailymail',
 'coco',
 'coco_captions',
 'coil100',
 'colorectal_histology',
 'colorectal_histology_large',
 'common_voice',
 'coqa',
 'cos_e',
 'cosmos_qa',
 'covid19',
 'covid19sum',
 'crema_d',
 'criteo',
 'cs_restaurants',
 'cura

### Download the Cifar10 dataset and view information

In [14]:
# Section 3.2

import tensorflow_datasets as tfds
import tensorflow.keras.backend as K

K.clear_session() # Making sure we are clearing out the TensorFlow graph

# Load a given dataset by name, along with the DatasetInfo
data, info = tfds.load("cifar10", with_info=True)
print(info)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/cifar10/3.0.2.incompleteQD0ELX/cifar10-train.tfrecord*...:   0%|          | 0/…

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/cifar10/3.0.2.incompleteQD0ELX/cifar10-test.tfrecord*...:   0%|          | 0/1…

Dataset cifar10 downloaded and prepared to ~/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.
tfds.core.DatasetInfo(
    name='cifar10',
    full_name='cifar10/3.0.2',
    description="""
    The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.
    """,
    homepage='https://www.cs.toronto.edu/~kriz/cifar.html',
    data_path='~/tensorflow_datasets/cifar10/3.0.2',
    file_format=tfrecord,
    download_size=162.17 MiB,
    dataset_size=132.40 MiB,
    features=FeaturesDict({
        'id': Text(shape=(), dtype=tf.string),
        'image': Image(shape=(32, 32, 3), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=50000, num_shards=1>,

### Exploring the data 

Here we will print the `data` and see what it provides. Then we will need to batch the data as data is provided as individual samples when you import it from `tensorflow-datasets`.

In [15]:
print(data)

{Split('train'): <PrefetchDataset element_spec={'id': TensorSpec(shape=(), dtype=tf.string, name=None), 'image': TensorSpec(shape=(32, 32, 3), dtype=tf.uint8, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None)}>, Split('test'): <PrefetchDataset element_spec={'id': TensorSpec(shape=(), dtype=tf.string, name=None), 'image': TensorSpec(shape=(32, 32, 3), dtype=tf.uint8, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None)}>}


In [16]:
# Print some training data
train_ds = data["train"].batch(16)
for item in train_ds:
    print(item)
    break

{'id': <tf.Tensor: shape=(16,), dtype=string, numpy=
array([b'train_16399', b'train_01680', b'train_47917', b'train_17307',
       b'train_27051', b'train_48736', b'train_26263', b'train_01456',
       b'train_19135', b'train_31598', b'train_12970', b'train_04223',
       b'train_27152', b'train_49635', b'train_04093', b'train_17537'],
      dtype=object)>, 'image': <tf.Tensor: shape=(16, 32, 32, 3), dtype=uint8, numpy=
array([[[[143,  96,  70],
         [141,  96,  72],
         [135,  93,  72],
         ...,
         [ 96,  37,  19],
         [105,  42,  18],
         [104,  38,  20]],

        [[128,  98,  92],
         [146, 118, 112],
         [170, 145, 138],
         ...,
         [108,  45,  26],
         [112,  44,  24],
         [112,  41,  22]],

        [[ 93,  69,  75],
         [118,  96, 101],
         [179, 160, 162],
         ...,
         [128,  68,  47],
         [125,  61,  42],
         [122,  59,  39]],

        ...,

        [[187, 150, 123],
         [184, 148, 

2022-07-29 18:45:21.326934: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [17]:
# Section 3.2

import tensorflow as tf

# Defining a dataset with batch size 16
train_ds = data["train"].batch(16)

# Creating a dataset that returns an (image, one-hot label) tuple
def format_data(x):
    return (x["image"], tf.one_hot(x["label"], depth=10))
train_ds = train_ds.map(format_data)

# Iterating the dataset
for item in train_ds:
    print(item)
    break

(<tf.Tensor: shape=(16, 32, 32, 3), dtype=uint8, numpy=
array([[[[143,  96,  70],
         [141,  96,  72],
         [135,  93,  72],
         ...,
         [ 96,  37,  19],
         [105,  42,  18],
         [104,  38,  20]],

        [[128,  98,  92],
         [146, 118, 112],
         [170, 145, 138],
         ...,
         [108,  45,  26],
         [112,  44,  24],
         [112,  41,  22]],

        [[ 93,  69,  75],
         [118,  96, 101],
         [179, 160, 162],
         ...,
         [128,  68,  47],
         [125,  61,  42],
         [122,  59,  39]],

        ...,

        [[187, 150, 123],
         [184, 148, 123],
         [179, 142, 121],
         ...,
         [198, 163, 132],
         [201, 166, 135],
         [207, 174, 143]],

        [[187, 150, 117],
         [181, 143, 115],
         [175, 136, 113],
         ...,
         [201, 164, 132],
         [205, 168, 135],
         [207, 171, 139]],

        [[195, 161, 126],
         [187, 153, 123],
         [186, 151

2022-07-29 18:45:21.386920: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


### Training a simple CNN on the Cifar10 data

In [18]:
# Section 3.2

from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.models import Sequential

# Defining a simple convolution neural network to process the CIFAR data
model = Sequential([
    Conv2D(64,(5,5), activation='relu', input_shape=(32,32,3)),
    Flatten(),
    Dense(10, activation='softmax')
])
# Compiling the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

# Fitting the model on the data for 25 epochs
model.fit(train_ds, epochs=25)

Epoch 1/25
3125/3125 [==============================] - 6s 2ms/step - loss: 4.4501 - acc: 0.1123
Epoch 2/25
3125/3125 [==============================] - 8s 2ms/step - loss: 2.2683 - acc: 0.1431
Epoch 3/25
3125/3125 [==============================] - 8s 3ms/step - loss: 2.2360 - acc: 0.1593
Epoch 4/25
3125/3125 [==============================] - 8s 3ms/step - loss: 2.2046 - acc: 0.1717
Epoch 5/25
3125/3125 [==============================] - 8s 2ms/step - loss: 2.1628 - acc: 0.1836
Epoch 6/25
3125/3125 [==============================] - 7s 2ms/step - loss: 2.1324 - acc: 0.1953
Epoch 7/25
3125/3125 [==============================] - 8s 2ms/step - loss: 2.1024 - acc: 0.2064
Epoch 8/25
3125/3125 [==============================] - 9s 3ms/step - loss: 2.0863 - acc: 0.2124
Epoch 9/25
3125/3125 [==============================] - 8s 3ms/step - loss: 2.0398 - acc: 0.2272
Epoch 10/25
3125/3125 [==============================] - 8s 2ms/step - loss: 2.0352 - acc: 0.2326
Epoch 11/25
3125/3125 [======